<a href="https://colab.research.google.com/github/Nedevi/APM1201/blob/main/FA7_Cobarrubias%2C_I%3B_Gaac%2C_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import time
import random

class HeapSort:
    def __init__(self, array):
        self.original_array = array
        self.in_place_sorted = []
        self.out_of_place_sorted = []

    # In-place heap sort
    def in_place_heap_sort(self):
        arr = self.original_array[:]
        n = len(arr)

        def heapify(arr, n, i):
            largest = i
            left = 2 * i + 1
            right = 2 * i + 2

            if left < n and arr[left] > arr[largest]:
                largest = left
            if right < n and arr[right] > arr[largest]:
                largest = right

            if largest != i:
                arr[i], arr[largest] = arr[largest], arr[i]
                heapify(arr, n, largest)

        # Build max-heap
        for i in range(n // 2 - 1, -1, -1):
            heapify(arr, n, i)

        # Extract elements from the heap
        for i in range(n - 1, 0, -1):
            arr[0], arr[i] = arr[i], arr[0]
            heapify(arr, i, 0)

        self.in_place_sorted = arr

    # Out-of-place heap sort (custom heap implementation)
    def out_of_place_heap_sort(self):
        arr = self.original_array[:]
        heap = []

        def heapify_up(heap, i):
            parent = (i - 1) // 2
            if i > 0 and heap[i] > heap[parent]:
                heap[i], heap[parent] = heap[parent], heap[i]
                heapify_up(heap, parent)

        def heapify_down(heap, i, n):
            largest = i
            left = 2 * i + 1
            right = 2 * i + 2

            if left < n and heap[left] > heap[largest]:
                largest = left
            if right < n and heap[right] > heap[largest]:
                largest = right

            if largest != i:
                heap[i], heap[largest] = heap[largest], heap[i]
                heapify_down(heap, largest, n)

        # Build the heap
        for value in arr:
            heap.append(value)
            heapify_up(heap, len(heap) - 1)

        # Extract elements from the heap
        sorted_arr = []
        while heap:
            heap[0], heap[-1] = heap[-1], heap[0]
            sorted_arr.append(heap.pop())
            heapify_down(heap, 0, len(heap))

        self.out_of_place_sorted = sorted_arr[::-1]  # Reverse to get ascending order

    def get_sorted_arrays(self):
        return self.in_place_sorted, self.out_of_place_sorted


class HeapSortComparison:
    @staticmethod
    def compare(input_sizes):
        results = []
        for size in input_sizes:
            array = [random.randint(1, 100000) for _ in range(size)]
            sorter = HeapSort(array)

            # In-place heap sort timing
            start = time.time()
            sorter.in_place_heap_sort()
            in_place_time = time.time() - start

            # Out-of-place heap sort timing
            start = time.time()
            sorter.out_of_place_heap_sort()
            out_of_place_time = time.time() - start

            results.append((size, in_place_time, out_of_place_time))

        return results

    @staticmethod
    def display_results(results):
        print("Comparison of In-Place and Out-of-Place Heap Sort (No heapq)")
        print(f"{'Input Size':<12}{'In-Place Sort (s)':<20}{'Out-of-Place Sort (s)':<20}")
        for size, in_place_time, out_of_place_time in results:
            print(f"{size:<12}{in_place_time:<20.6f}{out_of_place_time:<20.6f}")


# Run the comparison
input_sizes = [100, 1000, 5000, 10000, 50000]
results = HeapSortComparison.compare(input_sizes)
HeapSortComparison.display_results(results)



Comparison of In-Place and Out-of-Place Heap Sort (No heapq)
Input Size  In-Place Sort (s)   Out-of-Place Sort (s)
100         0.000522            0.000611            
1000        0.010313            0.009759            
5000        0.058099            0.062659            
10000       0.127109            0.139406            
50000       0.834051            0.632150            


In [9]:
import heapq

class StockOrderSystem:
    def __init__(self):
        self.buy_orders = []  # Max-heap for buy orders (store -price for max-heap behavior)
        self.sell_orders = [] # Min-heap for sell orders
        self.trade_log = []   # Log of completed trades

    def add_order(self, order_type, shares, price):
        """
        Adds a buy or sell order and processes matches if possible.
        :param order_type: "buy" or "sell"
        :param shares: Number of shares in the order
        :param price: Price per share in the order
        """
        if order_type == "buy":
            self._process_buy_order(shares, price)
        elif order_type == "sell":
            self._process_sell_order(shares, price)
        else:
            raise ValueError("Order type must be 'buy' or 'sell'.")

    def _process_buy_order(self, shares, price):
        print(f"New Buy Order: {shares} shares at ${price:.2f}")
        while shares > 0 and self.sell_orders and self.sell_orders[0][0] <= price:
            sell_price, sell_shares = heapq.heappop(self.sell_orders)
            if sell_shares > shares:
                # Partial match
                self.trade_log.append((shares, sell_price))
                print(f"Trade: {shares} shares at ${sell_price:.2f}")
                heapq.heappush(self.sell_orders, (sell_price, sell_shares - shares))
                shares = 0
            else:
                # Full match
                self.trade_log.append((sell_shares, sell_price))
                print(f"Trade: {sell_shares} shares at ${sell_price:.2f}")
                shares -= sell_shares
        if shares > 0:
            heapq.heappush(self.buy_orders, (-price, shares))

    def _process_sell_order(self, shares, price):
        print(f"New Sell Order: {shares} shares at ${price:.2f}")
        while shares > 0 and self.buy_orders and -self.buy_orders[0][0] >= price:
            buy_price, buy_shares = heapq.heappop(self.buy_orders)
            buy_price = -buy_price  # Convert back to positive
            if buy_shares > shares:
                # Partial match
                self.trade_log.append((shares, buy_price))
                print(f"Trade: {shares} shares at ${buy_price:.2f}")
                heapq.heappush(self.buy_orders, (-buy_price, buy_shares - shares))
                shares = 0
            else:
                # Full match
                self.trade_log.append((buy_shares, buy_price))
                print(f"Trade: {buy_shares} shares at ${buy_price:.2f}")
                shares -= buy_shares
        if shares > 0:
            heapq.heappush(self.sell_orders, (price, shares))

    def display_trade_log(self):
        """Prints all completed trades."""
        print("\nTrade Log:")
        if not self.trade_log:
            print("No trades have been executed.")
        else:
            for shares, price in self.trade_log:
                print(f"{shares} shares traded at ${price:.2f}")

    def display_order_books(self):
        """Displays the current buy and sell order books."""
        print("\nCurrent Buy Orders (Max-Heap):")
        for price, shares in sorted(self.buy_orders, reverse=True):
            print(f"{shares} shares at ${-price:.2f}")

        print("\nCurrent Sell Orders (Min-Heap):")
        for price, shares in self.sell_orders:
            print(f"{shares} shares at ${price:.2f}")

def main():
    system = StockOrderSystem()

    # Add a sequence of orders
    system.add_order("buy", 100, 50)    # Buy 100 shares at $50
    system.add_order("sell", 50, 40)    # Sell 50 shares at $40
    system.add_order("sell", 100, 60)   # Sell 100 shares at $60
    system.add_order("buy", 80, 55)     # Buy 80 shares at $55
    system.add_order("sell", 30, 45)    # Sell 30 shares at $45
    system.add_order("buy", 70, 60)     # Buy 70 shares at $60

    # Display the trade log and order books
    system.display_trade_log()
    system.display_order_books()

if __name__ == "__main__":
    main()


New Buy Order: 100 shares at $50.00
New Sell Order: 50 shares at $40.00
Trade: 50 shares at $50.00
New Sell Order: 100 shares at $60.00
New Buy Order: 80 shares at $55.00
New Sell Order: 30 shares at $45.00
Trade: 30 shares at $55.00
New Buy Order: 70 shares at $60.00
Trade: 70 shares at $60.00

Trade Log:
50 shares traded at $50.00
30 shares traded at $55.00
70 shares traded at $60.00

Current Buy Orders (Max-Heap):
50 shares at $50.00
50 shares at $55.00

Current Sell Orders (Min-Heap):
30 shares at $60.00
